In [1]:
# this will autoreload your packages (i.e. if you make a change to one of your .py files, you will not need to restart your notebook - your change is dynamically imported)
%load_ext autoreload  
%autoreload

In [2]:
import os
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" # to be able to make async database calls from the notebook
os.environ["DJANGO_SETTINGS_MODULE"] = "fpl.settings"
django.setup()

In [3]:
from fpl.models import FPLManager, Player, Event, Fixture, PointsInFixture, Team
from django.contrib.auth.models import User

fpl_manager, _ = FPLManager.objects.get_or_create(fpl_manager_id=6321588)

In [4]:
# dajemy ci playerow
for player_id in [101, 506, 114, 321, 504, 516, 526, 43, 326, 117, 60,524, 34, 602, 696]: ###DODAĆ KOD POBIERAJACY TEAM Z DANEGO GW
    fpl_manager.players.add(Player.objects.get(fpl_id=player_id))

In [5]:
fpl_manager.players.all()

<QuerySet [<Player: Mark Flekken (857)>, <Player: Alphonse Areola (927)>, <Player: Mads Roerslev Rasmussen (975)>, <Player: Alfie Doughty (1076)>, <Player: Issa Kaboré (1082)>, <Player: Murillo Santiago Costa dos Santos (1146)>, <Player: Pedro Porro (1167)>, <Player: Leon Bailey (1225)>, <Player: Douglas Luiz Soares de Paulo (1229)>, <Player: James Maddison (1524)>, <Player: Son Heung-min (1529)>, <Player: Jarrod Bowen (1534)>, <Player: Ollie Watkins (1573)>, <Player: Ivan Toney (1580)>, <Player: Carlton Morris (1626)>]>

In [6]:
from django.db.models import Q

# Tworzymy dummy data (to zajmie tak z 1 minute...)
for event in Event.objects.filter(Q(name="Gameweek 27") | Q(name="Gameweek 28")):
    for fixture in event.fixture_set.all():
        for player in Player.objects.filter(Q(team=fixture.team_h) | Q(team=fixture.team_a)):
            PointsInFixture.objects.update_or_create(player=player, fixture=fixture, defaults={"predicted_points": 2 if (player.id + fixture.id) % 2 == 0 else 5, # to nic nie znaczy - po prostu ranomowe liczby
                                                                                               "actual_points": 2 if (player.id + fixture.id) % 2 == 0 else 5})

In [28]:
# Zalozmy ze jestesmy w 27 kolejce i probujemy obczaic jakie transfery zrobic by zdobyc jak najwiecej punktow w 28 kolejce 

# Tyle punktow zdobylismy w 27 kolejce (jak na razie zakladamy ze kazdy zawodnik zagra tylko jeden mecz)
total_points_27 = 0
my_team = fpl_manager.players.all()
for player in my_team:
    total_points_27 += player.pointsinfixture_set.filter(fixture__event__name="Gameweek 27").first().actual_points
print(f"Tyle punktow zdobylismy w 27 kolejce: {total_points_27}")

# Jezeli zagramy taka sama druzyna w 28 kolejce to tyle punktow zdobedziemy (jak na razie zakladamy ze kazdy zawodnik zagra tylko jeden mecz)
total_points_28 = 0
for player in my_team:
    total_points_28 += player.pointsinfixture_set.filter(fixture__event__name="Gameweek 28").first().predicted_points
print(f"Jezeli zagramy taka sama druzyna w 28 kolejce to tyle punktow zdobedziemy: {total_points_28}")

# Wymienmy teraz Doughty na kogos innego (np. na Gabriela Osho)
my_team = my_team.exclude(first_name="Alfie", second_name="Doughty")
assert len(my_team) == 14
my_team = my_team | Player.objects.filter(first_name="Gabriel", second_name="Osho")
assert len(my_team) == 15

# Sprawdzmy ile punktow zdobedziemy w 28 kolejce po zmianie
total_points_28_after_transfer = 0
for player in my_team:
    total_points_28_after_transfer += player.pointsinfixture_set.filter(fixture__event__name="Gameweek 28").first().predicted_points
print(f"Jezeli zagramy taka sama druzyna w 28 kolejce po zmianie Doughty na Osho to tyle punktow zdobedziemy: {total_points_28_after_transfer}")

doughty_points = Player.objects.get(first_name="Alfie", second_name="Doughty").pointsinfixture_set.filter(fixture__event__name="Gameweek 28").first().predicted_points
osho_points = Player.objects.get(first_name="Gabriel", second_name="Osho").pointsinfixture_set.filter(fixture__event__name="Gameweek 28").first().predicted_points
print(f"To dlatego ze przewidujemy ze Doughty zdobedzie {doughty_points} punktow a Osho {osho_points} punktow")

##### PYTANIE: Jakiego zawodnika zamienic na jakiego zeby zdobyc jak najwiecej punktow w 28 kolejce? #####


Tyle punktow zdobylismy w 27 kolejce: 54.0
Jezeli zagramy taka sama druzyna w 28 kolejce to tyle punktow zdobedziemy: 63.0
Jezeli zagramy taka sama druzyna w 28 kolejce po zmianie Doughty na Osho to tyle punktow zdobedziemy: 60.0
To dlatego ze przewiduje ze Doughty zdobedzie 5.0 punktow a Osho 2.0 punktow


Replace None with None to score the most points in the 28th game week.
